In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [2]:
cv=LeaveOneOut()

In [3]:
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("label.csv",header=None)
y=np.array(y)
#print(X.shape)

In [4]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    param1=["best","random"]
    param_grid={"splitter":param1}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["splitter"]
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
    param_grid={"max_depth":param2}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["max_depth"]
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(10,i,50))  ## 设置max_features范围
    param_grid={"max_features":param3}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
#     print(datetime.datetime.now())
#     print("best parameters:{}".format(grid_search.best_params_))                        #输出最优网格化参数
#     print("best cross-validation score:{:.3f}".format(grid_search.best_score_))         #成员提供优化过程中观察到的最好评分
#     print("best estimator:\n{}".format(grid_search.best_estimator_))
    Predict_Y=[]    #存放预测值
    Prob_Y=[]       #存放预测概率
    Really_Y=y  #存放真实标签
    for train_index,test_index in cv.split(X):
        estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=grid_search.best_params_["max_features"])
        estimator.fit(X[train_index],y[train_index])
        Predict_Y.append(estimator.predict(X[test_index])[0])
        Prob_Y.append(estimator.predict_proba(X[test_index]))
    Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
    Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
    Prob_Y=pd.DataFrame(Prob_Y)
    Prob_Y.columns=["0","1"]
    confusion_matrix(Really_Y,Predict_Y)
    TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
    FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
    FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
    TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
    ACC=accuracy_score(Really_Y,Predict_Y)
    F1_score=f1_score(Really_Y,Predict_Y)
    AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
    Recall=recall_score(Really_Y,Predict_Y)
    Precision=precision_score(Really_Y,Predict_Y)
    S=p1
    D=p2
    M=grid_search.best_params_["max_features"]
#     print("ACC:",ACC)
#     print("f1_score:",F1_score)
#     print("AUC:",AUC)
    return TN,FN,FP,TP,ACC,Recall,Precision,F1_score,AUC,S,D,M

In [7]:
file_path = 'E:\datatest\PseDPC\d2\d21'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
211.csv
**************************************************
2022-02-24 15:05:18.065661
2022-02-24 15:05:19.603546
2022-02-24 15:05:26.649694
##################################################
212.csv
**************************************************
2022-02-24 15:05:28.215518
2022-02-24 15:05:29.688563
2022-02-24 15:05:36.464467
##################################################
213.csv
**************************************************
2022-02-24 15:05:37.904581
2022-02-24 15:05:39.533251
2022-02-24 15:05:47.464033
##################################################
214.csv
**************************************************
2022-02-24 15:05:48.977038
2022-02-24 15:05:50.610587
2022-02-24 15:05:58.492498
##################################################
215.csv
**************************************************
2022-02-24 15:06:00.042352
2022-02-24 15:06:01.659056
2022-02-24 15:06:09.524981
#############################################

In [8]:
file_path = 'E:\datatest\PseDPC\d2\d22'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
221.csv
**************************************************
2022-02-24 15:06:57.581563
2022-02-24 15:06:59.039686
2022-02-24 15:07:06.206485
##################################################
222.csv
**************************************************
2022-02-24 15:07:07.882999
2022-02-24 15:07:09.249343
2022-02-24 15:07:15.748953
##################################################
223.csv
**************************************************
2022-02-24 15:07:17.592024
2022-02-24 15:07:18.921466
2022-02-24 15:07:25.521832
##################################################
224.csv
**************************************************
2022-02-24 15:07:27.250182
2022-02-24 15:07:28.881845
2022-02-24 15:07:35.538008
##################################################
225.csv
**************************************************
2022-02-24 15:07:37.021062
2022-02-24 15:07:38.464179
2022-02-24 15:07:45.431538
#############################################

In [9]:
file_path = 'E:\datatest\PseDPC\d2\d23'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
231.csv
**************************************************
2022-02-24 15:08:16.666968
2022-02-24 15:08:18.165984
2022-02-24 15:08:25.190164
##################################################
232.csv
**************************************************
2022-02-24 15:08:27.186859
2022-02-24 15:08:28.509329
2022-02-24 15:08:34.697741
##################################################
233.csv
**************************************************
2022-02-24 15:08:37.277547
2022-02-24 15:08:38.937128
2022-02-24 15:08:46.962543
##################################################
234.csv
**************************************************
2022-02-24 15:08:48.765718
2022-02-24 15:08:50.392367
2022-02-24 15:08:58.439850
##################################################
235.csv
**************************************************
2022-02-24 15:08:59.915868
2022-02-24 15:09:01.377956
2022-02-24 15:09:08.572215
#############################################

In [10]:
file_path = 'E:\datatest\PseDPC\d2\d24'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
241.csv
**************************************************
2022-02-24 15:09:49.881452
2022-02-24 15:09:51.994797
2022-02-24 15:09:57.006842
##################################################
242.csv
**************************************************
2022-02-24 15:09:58.014147
2022-02-24 15:09:59.495184
2022-02-24 15:10:08.392421
##################################################
243.csv
**************************************************
2022-02-24 15:10:10.462875
2022-02-24 15:10:12.337859
2022-02-24 15:10:20.722426
##################################################
244.csv
**************************************************
2022-02-24 15:10:22.399960
2022-02-24 15:10:24.656949
2022-02-24 15:10:33.121323
##################################################
245.csv
**************************************************
2022-02-24 15:10:34.582414
2022-02-24 15:10:36.124267
2022-02-24 15:10:44.415086
#############################################

In [11]:
file_path = 'E:\datatest\PseDPC\d2\d25'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
251.csv
**************************************************
2022-02-24 15:11:33.625327
2022-02-24 15:11:35.206064
2022-02-24 15:11:42.941401
##################################################
252.csv
**************************************************
2022-02-24 15:11:44.777480
2022-02-24 15:11:46.370196
2022-02-24 15:11:54.983153
##################################################
253.csv
**************************************************
2022-02-24 15:11:57.050621
2022-02-24 15:11:58.987438
2022-02-24 15:12:07.894607
##################################################
254.csv
**************************************************
2022-02-24 15:12:09.893287
2022-02-24 15:12:11.563812
2022-02-24 15:12:19.844635
##################################################
255.csv
**************************************************
2022-02-24 15:12:21.435411
2022-02-24 15:12:23.090982
2022-02-24 15:12:31.813612
#############################################

In [12]:
file_path = 'E:\datatest\PseDPC\d2\d26'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
261.csv
**************************************************
2022-02-24 15:13:23.702853
2022-02-24 15:13:25.492067
2022-02-24 15:13:33.662603
##################################################
262.csv
**************************************************
2022-02-24 15:13:35.735058
2022-02-24 15:13:37.718751
2022-02-24 15:13:46.605973
##################################################
263.csv
**************************************************
2022-02-24 15:13:48.189735
2022-02-24 15:13:49.840319
2022-02-24 15:13:57.661394
##################################################
264.csv
**************************************************
2022-02-24 15:13:59.183322
2022-02-24 15:14:00.676349
2022-02-24 15:14:03.217556
##################################################
265.csv
**************************************************
2022-02-24 15:14:04.096177
2022-02-24 15:14:05.659994
2022-02-24 15:14:13.752343
#############################################

In [13]:
file_path = 'E:\datatest\PseDPC\d2\d27'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
271.csv
**************************************************
2022-02-24 15:15:01.146541
2022-02-24 15:15:02.835023
2022-02-24 15:15:09.532104
##################################################
272.csv
**************************************************
2022-02-24 15:15:11.327301
2022-02-24 15:15:12.981910
2022-02-24 15:15:21.310612
##################################################
273.csv
**************************************************
2022-02-24 15:15:22.763703
2022-02-24 15:15:24.401343
2022-02-24 15:15:32.263288
##################################################
274.csv
**************************************************
2022-02-24 15:15:34.058484
2022-02-24 15:15:35.734001
2022-02-24 15:15:44.548340
##################################################
275.csv
**************************************************
2022-02-24 15:15:46.666684
2022-02-24 15:15:48.267390
2022-02-24 15:15:57.065002
#############################################

In [14]:
file_path = 'E:\datatest\PseDPC\d2\d28'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
281.csv
**************************************************
2022-02-24 15:16:28.936752
2022-02-24 15:16:30.499571
2022-02-24 15:16:38.610869
##################################################
282.csv
**************************************************
2022-02-24 15:16:40.292370
2022-02-24 15:16:41.976864
2022-02-24 15:16:49.344152
##################################################
283.csv
**************************************************
2022-02-24 15:16:50.801254
2022-02-24 15:16:52.211481
2022-02-24 15:17:00.003634
##################################################
284.csv
**************************************************
2022-02-24 15:17:01.784868
2022-02-24 15:17:03.308823
2022-02-24 15:17:11.835290
##################################################
285.csv
**************************************************
2022-02-24 15:17:13.917719
2022-02-24 15:17:15.711919
2022-02-24 15:17:18.789705
#############################################

In [15]:
file_path = 'E:\datatest\PseDPC\d2\d29'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
291.csv
**************************************************
2022-02-24 15:17:43.257799
2022-02-24 15:17:44.776734
2022-02-24 15:17:53.899328
##################################################
292.csv
**************************************************
2022-02-24 15:17:55.745389
2022-02-24 15:17:57.369044
2022-02-24 15:18:06.425637
##################################################
293.csv
**************************************************
2022-02-24 15:18:08.473158
2022-02-24 15:18:10.280333
2022-02-24 15:18:18.256981
##################################################
294.csv
**************************************************
2022-02-24 15:18:20.159890
2022-02-24 15:18:21.942144
2022-02-24 15:18:24.506262
##################################################
295.csv
**************************************************
2022-02-24 15:18:25.159549
2022-02-24 15:18:26.869937
2022-02-24 15:18:29.390197
#############################################

In [16]:
file_path = 'E:\datatest\PseDPC\d2\d210'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
2101.csv
**************************************************
2022-02-24 15:19:03.180788
2022-02-24 15:19:05.099654
2022-02-24 15:19:14.696998
##################################################
2102.csv
**************************************************
2022-02-24 15:19:16.265812
2022-02-24 15:19:17.872481
2022-02-24 15:19:25.896035
##################################################
2103.csv
**************************************************
2022-02-24 15:19:27.508701
2022-02-24 15:19:29.275971
2022-02-24 15:19:32.464440
##################################################
2104.csv
**************************************************
2022-02-24 15:19:33.399937
2022-02-24 15:19:35.100400
2022-02-24 15:19:37.930836
##################################################
2105.csv
**************************************************
2022-02-24 15:19:39.102679
2022-02-24 15:19:40.710378
2022-02-24 15:19:49.843941
########################################

In [17]:
file_path = 'E:\datatest\PseDPC\d2\d211'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
2111.csv
**************************************************
2022-02-24 15:20:33.947041
2022-02-24 15:20:35.613645
2022-02-24 15:20:43.717962
##################################################
2112.csv
**************************************************
2022-02-24 15:20:45.332670
2022-02-24 15:20:47.241535
2022-02-24 15:20:56.514752
##################################################
2113.csv
**************************************************
2022-02-24 15:20:58.087516
2022-02-24 15:20:59.679279
2022-02-24 15:21:08.130646
##################################################
2114.csv
**************************************************
2022-02-24 15:21:10.103368
2022-02-24 15:21:11.782895
2022-02-24 15:21:14.542491
##################################################
2115.csv
**************************************************
2022-02-24 15:21:15.461033
2022-02-24 15:21:17.284156
2022-02-24 15:21:26.326966
########################################

In [18]:
file_path = 'E:\datatest\PseDPC\d2\d212'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
2121.csv
**************************************************
2022-02-24 15:22:09.520556
2022-02-24 15:22:10.510906
2022-02-24 15:22:12.750935
##################################################
2122.csv
**************************************************
2022-02-24 15:22:13.088012
2022-02-24 15:22:14.054426
2022-02-24 15:22:15.919458
##################################################
2123.csv
**************************************************
2022-02-24 15:22:16.290443
2022-02-24 15:22:17.933048
2022-02-24 15:22:26.764443
##################################################
2124.csv
**************************************************
2022-02-24 15:22:28.597516
2022-02-24 15:22:30.541316
2022-02-24 15:22:40.238599
##################################################
2125.csv
**************************************************
2022-02-24 15:22:42.140488
2022-02-24 15:22:44.157093
2022-02-24 15:22:47.354561
########################################

In [19]:
file_path = 'E:\datatest\PseDPC\d2\d213'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
2131.csv
**************************************************
2022-02-24 15:23:11.405191
2022-02-24 15:23:13.290148
2022-02-24 15:23:22.706953
##################################################
2132.csv
**************************************************
2022-02-24 15:23:24.695632
2022-02-24 15:23:26.793054
2022-02-24 15:23:30.025373
##################################################
2133.csv
**************************************************
2022-02-24 15:23:30.895046
2022-02-24 15:23:32.828872
2022-02-24 15:23:42.893943
##################################################
2134.csv
**************************************************
2022-02-24 15:23:44.553513
2022-02-24 15:23:46.887262
2022-02-24 15:23:56.092630
##################################################
2135.csv
**************************************************
2022-02-24 15:23:57.706313
2022-02-24 15:23:59.528459
2022-02-24 15:24:08.752841
########################################

In [20]:
file_path = 'E:\datatest\PseDPC\d2\d214'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
2141.csv
**************************************************
2022-02-24 15:24:46.861901
2022-02-24 15:24:48.883472
2022-02-24 15:24:58.153043
##################################################
2142.csv
**************************************************
2022-02-24 15:24:59.874437
2022-02-24 15:25:01.595831
2022-02-24 15:25:10.425209
##################################################
2143.csv
**************************************************
2022-02-24 15:25:11.936188
2022-02-24 15:25:13.767294
2022-02-24 15:25:23.063396
##################################################
2144.csv
**************************************************
2022-02-24 15:25:24.676084
2022-02-24 15:25:26.493252
2022-02-24 15:25:35.370500
##################################################
2145.csv
**************************************************
2022-02-24 15:25:36.865468
2022-02-24 15:25:38.675625
2022-02-24 15:25:47.915902
########################################

In [21]:
file_path = 'E:\datatest\PseDPC\d2\d215'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
2151.csv
**************************************************
2022-02-24 15:26:45.446978
2022-02-24 15:26:47.329941
2022-02-24 15:26:56.767690
##################################################
2152.csv
**************************************************
2022-02-24 15:26:58.548957
2022-02-24 15:27:00.485743
2022-02-24 15:27:09.469706
##################################################
2153.csv
**************************************************
2022-02-24 15:27:11.261933
2022-02-24 15:27:13.116948
2022-02-24 15:27:21.975247
##################################################
2154.csv
**************************************************
2022-02-24 15:27:24.162396
2022-02-24 15:27:26.192963
2022-02-24 15:27:29.976866
##################################################
2155.csv
**************************************************
2022-02-24 15:27:30.886405
2022-02-24 15:27:32.760394
2022-02-24 15:27:36.235116
########################################

In [22]:
file_path = 'E:\datatest\PseDPC\d2\d216'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
2161.csv
**************************************************
2022-02-24 15:28:25.287854
2022-02-24 15:28:27.132918
2022-02-24 15:28:35.834658
##################################################
2162.csv
**************************************************
2022-02-24 15:28:37.506165
2022-02-24 15:28:39.373170
2022-02-24 15:28:47.624115
##################################################
2163.csv
**************************************************
2022-02-24 15:28:49.255730
2022-02-24 15:28:51.059902
2022-02-24 15:28:59.892271
##################################################
2164.csv
**************************************************
2022-02-24 15:29:01.589729
2022-02-24 15:29:03.315113
2022-02-24 15:29:12.266165
##################################################
2165.csv
**************************************************
2022-02-24 15:29:13.786098
2022-02-24 15:29:15.429723
2022-02-24 15:29:24.677958
########################################

In [23]:
file_path = 'E:\datatest\PseDPC\d2\d217'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
2171.csv
**************************************************
2022-02-24 15:30:19.246686
2022-02-24 15:30:20.951126
2022-02-24 15:30:29.383103
##################################################
2172.csv
**************************************************
2022-02-24 15:30:30.979810
2022-02-24 15:30:32.833850
2022-02-24 15:30:42.554841
##################################################
2173.csv
**************************************************
2022-02-24 15:30:44.397931
2022-02-24 15:30:46.132270
2022-02-24 15:30:55.171086
##################################################
2174.csv
**************************************************
2022-02-24 15:30:56.793745
2022-02-24 15:30:58.538099
2022-02-24 15:31:07.179985
##################################################
2175.csv
**************************************************
2022-02-24 15:31:08.697895
2022-02-24 15:31:10.360448
2022-02-24 15:31:19.326459
########################################

In [24]:
file_path = 'E:\datatest\PseDPC\d2\d218'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
2181.csv
**************************************************
2022-02-24 15:32:13.964459
2022-02-24 15:32:15.808526
2022-02-24 15:32:26.073063
##################################################
2182.csv
**************************************************
2022-02-24 15:32:27.803454
2022-02-24 15:32:29.538790
2022-02-24 15:32:38.276412
##################################################
2183.csv
**************************************************
2022-02-24 15:32:39.838234
2022-02-24 15:32:41.829905
2022-02-24 15:32:50.876700
##################################################
2184.csv
**************************************************
2022-02-24 15:32:53.337117
2022-02-24 15:32:55.111371
2022-02-24 15:33:04.353427
##################################################
2185.csv
**************************************************
2022-02-24 15:33:06.567483
2022-02-24 15:33:08.427527
2022-02-24 15:33:17.780503
########################################

In [25]:
file_path = 'E:\datatest\PseDPC\d2\d219'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
2191.csv
**************************************************
2022-02-24 15:34:03.204948
2022-02-24 15:34:04.915371
2022-02-24 15:34:13.170286
##################################################
2192.csv
**************************************************
2022-02-24 15:34:14.663291
2022-02-24 15:34:16.319884
2022-02-24 15:34:24.362342
##################################################
2193.csv
**************************************************
2022-02-24 15:34:25.911198
2022-02-24 15:34:27.528869
2022-02-24 15:34:35.963303
##################################################
2194.csv
**************************************************
2022-02-24 15:34:37.508170
2022-02-24 15:34:39.563721
2022-02-24 15:34:42.222579
##################################################
2195.csv
**************************************************
2022-02-24 15:34:43.005485
2022-02-24 15:34:44.780767
2022-02-24 15:34:53.361808
########################################

In [26]:
file_path = 'E:\datatest\PseDPC\d2\d220'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
2201.csv
**************************************************
2022-02-24 15:35:36.186200
2022-02-24 15:35:37.765974
2022-02-24 15:35:40.607371
##################################################
2202.csv
**************************************************
2022-02-24 15:35:41.522921
2022-02-24 15:35:43.129648
2022-02-24 15:35:51.293780
##################################################
2203.csv
**************************************************
2022-02-24 15:35:52.994230
2022-02-24 15:35:54.607914
2022-02-24 15:36:02.634437
##################################################
2204.csv
**************************************************
2022-02-24 15:36:04.222222
2022-02-24 15:36:05.810939
2022-02-24 15:36:13.705844
##################################################
2205.csv
**************************************************
2022-02-24 15:36:15.440176
2022-02-24 15:36:17.067821
2022-02-24 15:36:25.054453
########################################